In [1]:
# from model.rgcn2 import GNNPolicy
from model.tripartite import GNNPolicy

from model.tripartite_lp import GNNPolicy as Trimodel
import os
import torch
from torch_geometric.data import HeteroData
import collections
from uc_class import UC
import numpy as np
import json
from utilities import *
import copy

In [2]:

DEVICE = 'cpu'

instance = '80_c11_based_8_std'
model = 'tripartite'


tri_path = 'model_save/1bin_80_c11_based_8_std_tripartite_lpsolve/model_best_acc.pth'
tri_model = Trimodel()
tri_dict = torch.load(tri_path)
tri_model.load_state_dict(tri_dict)
tri_model.eval()


model_save_path = f'./model_save/local_search_{instance}_{model}/'
PredictModel = GNNPolicy().to(DEVICE)
if os.path.exists(model_save_path+'model_best.pth'):
    state_dict = torch.load(model_save_path+'model_best.pth')
    PredictModel.load_state_dict(state_dict, strict=False)




print("载入模型成功")

载入模型成功


In [3]:
uc_path = f'/home/peilun/predict_heuristicFix_loopEnhance/UC_AF/{instance}.mod'
uc = UC(uc_path)

shut_down = uc.Pishutdown
startup = uc.Pistartup
on = uc.ThTime_on_min
off = uc.ThTime_off_min
pmax = uc.ThPimax
pmin = uc.ThPimin
ramp_up = uc.Piup
ramp_down = uc.Pidown


In [4]:
# json_test_path = f'./datasets/{instance}/total_json/test'
# test_files = os.listdir(json_test_path)[:1]
# for file in test_files:
#     with open(os.path.join(json_test_path, file),'r') as f:
#         data = json.load(f)
#     print(file)
#     pmax = uc.ThPimax
#     Dt = data['Dt']
#     Spin = data['Spin']
#     u0 = data['u0']
#     p0 = data['p0']
#     on_off = data['on_off']
#     m = uc.get_1bin_model(Dt=Dt,Spin=Spin,ThTime_on_off_init=on_off,Ui0=u0,Pi0=p0)
#     Tri_graph=get_Tripartite_graph_lp(m)
#     sol = tri_model(Tri_graph).detach().cpu().numpy().reshape(-1,24)
#     sol = restore(sol,uc,data).reshape(-1).astype(int)
#     predict_obj = solve(m,sol)

#     search_graph = get_local_search_graph(m,torch.from_numpy(sol))
#     neighbor = PredictModel(search_graph).detach().cpu().numpy().reshape(-1)
#     threshold_num = np.percentile(neighbor, 95)
#     local = np.where(neighbor > threshold_num, 1, 0)
#     print(np.count_nonzero(local))
#     search_obj = local_search(m,sol,local)
#     opt_obj = data['obj']
#     print((predict_obj-search_obj)/(predict_obj-opt_obj))


In [5]:
def local_search_repeat(m,sol,graph):
    Tri_graph = copy.deepcopy(graph)
    for v in ['u','p','s']:
        Tri_graph[v].x = torch.cat((Tri_graph[v].x, torch.zeros(Tri_graph[v].x.shape[0]).reshape(-1,1)), dim=1)
    last=solve(m,sol)
    
    # print(last)
    cnt=0
    while True:
        u_sol,p_sol,s_sol = get_solutions(m,sol)
        sol_dict = {'u': u_sol, 'p': p_sol, 's': s_sol}  # u_sol, p_sol, s_sol 是对应的解
        for v in ['u','p','s']:
            Tri_graph[v].x[..., -1] = sol_dict[v].reshape(-1)
        neighbor = PredictModel(Tri_graph).detach().cpu().numpy().reshape(-1)
        threshold_num = np.percentile(neighbor, 85)
        local = np.where(neighbor > threshold_num, 1, 0)
        new,sol = local_search(m,sol,local)
        # print(new)
        cnt+=1
        if (last-new)/last<=1e-6:
            break
        last=new
    return new,cnt

In [6]:
json_test_path = f'./datasets/{instance}/total_json/test'
test_files = os.listdir(json_test_path)[::10]
for file in test_files:
    with open(os.path.join(json_test_path, file),'r') as f:
        data = json.load(f)
    # print(file)
    pmax = uc.ThPimax
    Dt = data['Dt']
    Spin = data['Spin']
    u0 = data['u0']
    p0 = data['p0']
    on_off = data['on_off']
    m = uc.get_1bin_model(Dt=Dt,Spin=Spin,ThTime_on_off_init=on_off,Ui0=u0,Pi0=p0)
    Tri_graph=get_Tripartite_graph_lp(m)
    sol = tri_model(Tri_graph).detach().cpu().numpy().reshape(-1,24)
    sol = restore(sol,uc,data).reshape(-1).astype(int)
    last=solve(m,sol)
    print(file)
    print('before_gap',(last-data['obj'])/last)
    obj,loop_times = local_search_repeat(m,sol,Tri_graph)
    print('loop_times',loop_times)
    print('after_gap',(obj-data['obj'])/obj)
    print('\n')

107_1.json
before_gap 0.013828369676359581
loop_times 2
after_gap 0.004557968699600072


107_18.json
before_gap 0.016446580199159157
loop_times 2
after_gap 0.006721569866316468


117_1.json
before_gap 0.008012775956738792
loop_times 2
after_gap 0.002532131291931764


117_18.json
before_gap 0.008219305129244984
loop_times 2
after_gap 0.005728022479809997


127_0.json
before_gap 0.006530015428730449
loop_times 2
after_gap 0.005973965613883369


127_18.json
before_gap 0.004379400558380598
loop_times 2
after_gap 0.0041706816766034


137_0.json
before_gap 0.008624161936663833
loop_times 2
after_gap 0.008254919415613467


137_18.json
before_gap 0.00800953889670893
loop_times 2
after_gap 0.00667760331493482


137_9.json
before_gap 0.011137632320795282
loop_times 2
after_gap 0.010587121416293742


147_19.json
before_gap 0.012674107457265884
loop_times 2
after_gap 0.004742605380618496


157_0.json
before_gap 0.004505783242925523
loop_times 2
after_gap 0.003692163719550652


157_16.json
before_g